In [1]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc
)
from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()
from pymystem3 import Mystem
mystem = Mystem()
import spacy
import flair
import nltk
from string import punctuation
punctuation = punctuation + '—–“”«»'
from sklearn.metrics import accuracy_score
import re
from flair.models import SequenceTagger

ModuleNotFoundError: No module named 'flair'

In [7]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2020-10-14 20:41:10--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.243, 5.45.205.245, 5.45.205.241, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.243|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm904.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-10-14 20:41:11--  http://cache-mskm904.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm904.cdn.yandex.net (cache-mskm904.cdn.yandex.net)... 5.45.220.14, 2a02:6b8:0:2002::15
Connecting to cache-mskm904.cdn.yandex.net (cache-mskm904.cdn.yandex.net)|5.45.220.14|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz’

mystem-3.0-linux3.1 100%[===================>]  1

In [8]:
#причастие - VERB
#аббревиатура - NOUN
#местоимение - PRON

In [9]:
rus_texts = ['НКРЯ - мой самый любимый корпус.', 'Собака, пробегавшая мимо, зачем-то начала яростно гавкать.', 'Мой друг Петр говорит по-русски очень плохо.', 
             'Я люблю готовить, слушая музыку.', 'Вчера в Сан-Франциско шел ливень.', 'Я был там две недели назад и видел ели.', 'У нее темно-синее платье.', 
             'Я набрал семьдесят четыре балла по матеше.', 'Я купил пурпурную портативную зарядку.', 'Из-за него мы опоздали на концерт.', 'Я люблю есть люля-кебаб.',
             'Выпив чай, я отправился на прогулку и убрал айпад.', 'А расширение границ прямо противоречит международным договорам, соглашениям, документам ООН.', 
             'Перед сном из блокнота всё переписал набело.', 'Вчера мой отец завел новый фотосправочник.', 'Это всё из-за моего бывшего.', 'Чем бы дитя не тешилось, лишь бы не плакало.', 
             'Марианна была ультраженственной в этот вечер.', 'Вася, смотри как я висю на ветке!', 'Несомненно, это была тюрбо.', 'Он написал этот текст ручками с грезитвой.']
rus_tags = ['NOUN PRON PRON ADJ NOUN', 'NOUN VERB ADV ADV VERB ADV VERB', 'PRON NOUN NOUN VERB ADV ADV ADV',
            'PRON VERB VERB VERB NOUN', 'ADV PREP NOUN VERB NOUN', 'PRON VERB PRON NUM NOUN ADV CONJ VERB NOUN', 'PREP PRON ADJ NOUN',
            'PRON VERB NUM NUM NOUN PREP NOUN', 'PRON VERB ADJ ADJ NOUN', 'PREP PRON PRON VERB PREP NOUN', 'PRON VERB VERB NOUN',
            'VERB NOUN PRON VERB PREP NOUN CONJ VERB NOUN', 'CONJ NOUN NOUN ADV VERB ADJ NOUN NOUN NOUN NOUN', 
            'PREP NOUN PREP NOUN PRON VERB ADV', 'ADV PRON NOUN VERB ADJ NOUN', 'PRON PRON PREP PRON ADJ', 'CONJ PART NOUN PART VERB PART PART PART VERB',
            'NOUN VERB ADJ PREP PRON NOUN', 'NOUN VERB CONJ PRON VERB PREP NOUN', 'ADV PRON VERB NOUN', 'PRON VERB PRON NOUN NOUN CONJ NOUN']

In [27]:
eng_texts = ['I have a lot of gold.', 'His abduction and imprisonment of Pamela, his over-riding jealousy of Mr. Williams and later of the child Pamela is about to bear him.',
             'Every morning Mary goes for a run.', 'Paul enjoys a drink.', 'It is mine!', 'Yesterday I bought three oranges, it was the best decision of the day.',
             'I buy green very often, it is good for my health.', 'She was beautiful, that tall, skinny, twenty-year-old girl.', 'I goed to the cinema with Mark yesterday.', 
             'Mr Lacayo said UNO would receive two million dollars of the US aid.', 
             'Thereza Imanishi-Kari, who was then an immunologist at the Massachusetts Institute of Technology (MIT).', 'Felix who will live in the centre with Bro.']
eng_tags = ['NOUN VERB ART ADV PREP NOUN', 'PRON NOUN CONJ NOUN PREP NOUN PRON ADJ NOUN PREP NOUN NOUN CONJ NOUN PREP ART NOUN NOUN VERB ADV PREP VERB PRON',
            'ADJ NOUN NOUN VERB PREP ART NOUN', 'NOUN VERB ART NOUN', 'PRON VERB PRON', 'ADV PRON VERB NUM NOUN PRON VERB ART ADJ NOUN PREP ART NOUN',
            'PRON VERB NOUN ADV ADV PRON VERB ADV PREP PRON NOUN', 'PRON VERB ADJ PRON ADJ ADJ ADJ NOUN', 'PRON VERB PREP ART NOUN CONJ NOUN ADV',
            'NOUN NOUN VERB NOUN VERB VERB NUM NUM NOUN PREP ART ADJ NOUN', 
            'NOUN NOUN PRON VERB PRON ART NOUN PREP ART NOUN NOUN PREP NOUN NOUN', 'NOUN PRON VERB VERB PREP ART NOUN CONJ NOUN']

In [11]:
def clean(texts_list, tags_list):
  clean_list = []
  new_tags_list = []
  for i in texts_list:
    i = i.lower()
    for p in punctuation:
      if p in i:
        i = i.replace(p, '')
        i = i.replace('  ', ' ')
    i = i.split(' ')
    for n in i:
      clean_list.append(n)
    new_tags_list = []
  for tag in tags_list:
    tag = tag.split(' ')
    for t in tag:
      new_tags_list.append(t)
  return clean_list, new_tags_list

In [12]:
good = clean(rus_texts, rus_tags)

In [28]:
good_eng = clean(eng_texts, eng_tags)

## **PYMORPHY2**

In [13]:
pymorphy_tags = []
for i in good[0]:
  parse = m.parse(i)[0].tag.POS
  pymorphy_tags.append(parse)

In [14]:
new_pymorphy_tags = []
for p in pymorphy_tags:
  p = p.replace('PRCL', 'PART')
  p = p.replace('ADJF', 'ADJ')
  p = p.replace('ADJS', 'ADJ')
  p = p.replace('ADVB', 'ADV')
  p = p.replace('COMP', 'ADJ')
  p = p.replace('GRND', 'VERB')
  p = p.replace('INFN', 'VERB')
  p = p.replace('PRTF', 'VERB')
  p = p.replace('PRTS', 'VERB')
  p = p.replace('NPRO', 'PRON')
  p = p.replace('NUMR', 'NUM')
  p = p.replace('PRED', 'ADV')
  new_pymorphy_tags.append(p)

In [15]:
accuracy_score(good[1], new_pymorphy_tags)

0.835820895522388

## **NATASHA**

In [16]:
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

In [17]:
natasha_tags = []
n = ''
for i in good[0]:
  n += i + ' '
doc = Doc(n)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
for d in doc.tokens:
  natasha_tags.append(d.pos)

In [25]:
new_natasha_tags = []
for n in natasha_tags:
  n = re.sub('^AUX$', 'VERB', n)
  n = re.sub('^SCONJ$', 'CONJ', n)
  n = re.sub('^ADP$', 'PREP', n)
  n = re.sub('^CCONJ$', 'CONJ', n)
  new_natasha_tags.append(n)

In [26]:
accuracy_score(good[1], new_natasha_tags)

0.835820895522388

## **PYMYSTEM3**

In [18]:
mystem_tags = []
for i in good[0]:
  m = mystem.analyze(i)
  gr = m[0]['analysis'][0]['gr']
  pos = gr.split('=')[0].split(',')[0]
  mystem_tags.append(pos)

In [19]:
new_mystem_tags = []
for mt in mystem_tags:
  mt = re.sub('^ADVPRO$', 'ADV', mt)
  mt = re.sub('^ANUM$', 'NUM', mt)
  mt = re.sub('^APRO$', 'PRON', mt)
  mt = re.sub('^SPRO$', 'PRON', mt)
  mt = re.sub('^PR$', 'PREP', mt)
  mt = re.sub('^S$', 'NOUN', mt)
  mt = re.sub('^V$', 'VERB', mt)
  mt = re.sub('^A$', 'ADJ', mt)
  new_mystem_tags.append(mt)

In [20]:
accuracy_score(good[1], new_mystem_tags)

0.9104477611940298

## **SpaCy**

In [37]:
spacy_tags = []
nlp = spacy.load("en_core_web_sm")
s = ''
for i in good_eng[0]:
  s += i + ' '
doc = nlp(s)
for token in doc:
    spacy_tags.append(token.pos_)

In [41]:
new_spacy_tags = []
for sp in spacy_tags:
  sp = re.sub('^PROPN$', 'PRON', sp)
  sp = re.sub('^ADP$', 'PREP', sp)
  sp = re.sub('^AUX$', 'VERB', sp)
  sp = re.sub('^SCONJ$', 'CONJ', sp)
  sp = re.sub('^CCONJ$', 'CONJ', sp)
  new_spacy_tags.append(sp)

In [42]:
accuracy_score(good_eng[1], new_spacy_tags)

0.6050420168067226

## **Flair**

In [52]:
tagger = SequenceTagger.load('pos-multi')
tagger = SequenceTagger.load('ner')

2020-10-14 23:08:58,611 loading file /root/.flair/models/pos-multi-v0.1.pt
2020-10-14 23:08:59,835 https://nlp.informatik.hu-berlin.de/resources/models/ner/en-ner-conll03-v0.4.pt not found in cache, downloading to /tmp/tmpc6mcwaxi


100%|██████████| 432197603/432197603 [00:18<00:00, 23224394.28B/s]

2020-10-14 23:09:18,806 copying /tmp/tmpc6mcwaxi to cache at /root/.flair/models/en-ner-conll03-v0.4.pt


2020-10-14 23:09:20,973 removing temp file /tmp/tmpc6mcwaxi
2020-10-14 23:09:21,044 loading file /root/.flair/models/en-ner-conll03-v0.4.pt


In [54]:
sentence = Sentence(s)

NameError: ignored